In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 # 1. 事前に準備済みのK8s環境を確認

In [98]:
%%bash
az aks show -g rg_bdc -n o9o9aks

Name     Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
-------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------
o9o9aks  japaneast   rg_bdc           1.14.6               Succeeded            o9o9aks-rgbdc-745830-7ebfd297.hcp.japaneast.azmk8s.io


In [99]:
%%bash
az vm list -g MC_RG_BDC_O9O9AKS_JAPANEAST -d

Name                      ResourceGroup                PowerState    PublicIps    Fqdns    Location    Zones
------------------------  ---------------------------  ------------  -----------  -------  ----------  -------
aks-nodepool1-15232104-0  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-1  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-2  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast


In [100]:
%%bash
kubectl get node -o wide



NAME                       STATUS     ROLES   AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-15232104-0   Ready      agent   73d   v1.14.6   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-1   Ready      agent   73d   v1.14.6   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-2   NotReady   agent   73d   v1.14.6   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6


 # 2. シンプルデモ（エンジンエックス）
 ![](image/2019-12-03-12-54-07.png)

 ## 2−1. namespace作成

In [104]:
%%bash
kubectl create namespace hello-k8s


namespace/hello-k8s created


 ## 2−2. Pod作成(nginxコンテナを含んだPodを３冗長で作成しラベルmyappをつける)

In [106]:
%%bash
kubectl run myapp --image=nginx:1.17 --replicas 3 --labels="app=myapp" -n hello-k8s


deployment.apps/myapp created
kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.


In [107]:
%%bash
kubectl get pod -o wide -n hello-k8s

NAME                     READY   STATUS    RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
myapp-7c54b65df9-78hk8   1/1     Running   0          18s   10.244.0.98    aks-nodepool1-15232104-1   <none>           <none>
myapp-7c54b65df9-csdds   1/1     Running   0          18s   10.244.2.119   aks-nodepool1-15232104-0   <none>           <none>
myapp-7c54b65df9-vsqwz   1/1     Running   0          18s   10.244.0.99    aks-nodepool1-15232104-1   <none>           <none>


 ## 2−3. Service作成（ホストポート80番で受けてラベルmyappに転送するロードバランサを作成）

In [108]:
%%bash
kubectl create service loadbalancer --tcp 80:80 myapp -n hello-k8s


service/myapp created


In [109]:
%%bash
kubectl get service myapp -n hello-k8s


NAME    TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
myapp   LoadBalancer   10.0.251.140   52.246.164.47   80:31061/TCP   3m46s


 ## 2−4. サービスにアクセス
 http://[EXTERNAL-IP]
 ![](image/2019-12-03-13-29-19.png)

 ## 2−5. namespace削除

In [110]:
%%bash
kubectl delete namespace hello-k8s


namespace "hello-k8s" deleted


 # 3. マニュフェストによるリソース作成

 # 3−1. デモ用Namespace作成

In [112]:
%%bash
kubectl create namespace basic-k8s


namespace/basic-k8s created


 # 3−2. Pod作成
 ![](image/2019-12-03-13-05-35.png)

In [113]:
%%bash
kubectl get pod -o wide -n basic-k8s

No resources found in basic-k8s namespace.


In [114]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/deployment.yaml

# A. 基本項目
apiVersion: apps/v1
kind: Deployment
metadata:
  name: photoview-deployment

# B. Deploymentのスペック
spec:
  replicas: 5   # レプリカ数
  selector:
    matchLabels:
      app: photo-view   # テンプレートの検索条件

  # C. Podのテンプレート
  template:
    metadata:
      labels:
        app: photo-view
        env: stage
    spec:
      containers:
      - image: o9o9acr.azurecr.io/photo-view:v1.0   # コンテナイメージの場所
        name: photoview-container   # コンテナ名
        ports:
        - containerPort: 80   # ポート番号


In [115]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/deployment.yaml -n basic-k8s

deployment.apps/photoview-deployment created


In [116]:
%%bash
kubectl get pod -o wide -n basic-k8s

NAME                                   READY   STATUS    RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
photoview-deployment-5b6f5dbdf-6jrx4   1/1     Running   0          13s   10.244.0.100   aks-nodepool1-15232104-1   <none>           <none>
photoview-deployment-5b6f5dbdf-cv9pl   1/1     Running   0          13s   10.244.0.101   aks-nodepool1-15232104-1   <none>           <none>
photoview-deployment-5b6f5dbdf-dnzgt   1/1     Running   0          13s   10.244.2.122   aks-nodepool1-15232104-0   <none>           <none>
photoview-deployment-5b6f5dbdf-pnkkf   1/1     Running   0          13s   10.244.2.120   aks-nodepool1-15232104-0   <none>           <none>
photoview-deployment-5b6f5dbdf-wh677   1/1     Running   0          13s   10.244.2.121   aks-nodepool1-15232104-0   <none>           <none>


 ![](image/2019-12-03-13-06-57.png)

 # 3−3. Service作成

In [117]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/service.yaml


# A. 基本項目
apiVersion: v1
kind: Service
metadata:
  name: webserver

# B. Serviceのスペック
spec:
  type: LoadBalancer
  ports:   # ポート番号
    - port: 80
      targetPort: 80
      protocol: TCP
  
  # C. Podの条件(ラベル)
  selector:
    app: photo-view


In [118]:
%%bash
kubectl apply -f ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/service.yaml -n basic-k8s

service/webserver created


In [119]:
%%bash
kubectl get service -o wide -n basic-k8s

NAME        TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE   SELECTOR
webserver   LoadBalancer   10.0.137.214   13.71.128.107   80:30844/TCP   72s   app=photo-view


 ![](image/2019-12-03-13-07-50.png)

 # 3−4. サービスにアクセス
 http://[EXTERNAL-IP]
 ![](image/2019-12-03-13-08-54.png)
 ![](image/2019-12-03-13-36-16.png)

 # 3−5. Pod擬似障害

In [120]:
%%bash
kubectl delete pod photoview-deployment-5b6f5dbdf-6jrx4 -n basic-k8s


pod "photoview-deployment-5b6f5dbdf-6jrx4" deleted


In [122]:
%%bash
kubectl get pod -o wide -n basic-k8s



NAME                                   READY   STATUS    RESTARTS   AGE    IP             NODE                       NOMINATED NODE   READINESS GATES
photoview-deployment-5b6f5dbdf-5xl7x   1/1     Running   0          44s    10.244.0.102   aks-nodepool1-15232104-1   <none>           <none>
photoview-deployment-5b6f5dbdf-cv9pl   1/1     Running   0          4m8s   10.244.0.101   aks-nodepool1-15232104-1   <none>           <none>
photoview-deployment-5b6f5dbdf-dnzgt   1/1     Running   0          4m8s   10.244.2.122   aks-nodepool1-15232104-0   <none>           <none>
photoview-deployment-5b6f5dbdf-pnkkf   1/1     Running   0          4m8s   10.244.2.120   aks-nodepool1-15232104-0   <none>           <none>
photoview-deployment-5b6f5dbdf-wh677   1/1     Running   0          4m8s   10.244.2.121   aks-nodepool1-15232104-0   <none>           <none>


 # 3−6. 環境破棄

In [123]:
%%bash
kubectl delete -f ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/service.yaml -n basic-k8s
kubectl delete -f ~/Jupyter/SQL_Server_on_K8s/Basic/Understanding-K8s/chap03/deployment.yaml -n basic-k8s


service "webserver" deleted
deployment.apps "photoview-deployment" deleted


 ![](image/2019-12-03-13-05-35.png)